# Using MIMIC IV with python
## Basic example

This notebook will show you a simple example of how to connect to the MIMIC IV database and execute queries on it.
Results are returned as pandas dataframes.

Notes: 
- Tables are organized as shown in the [documentation](https://mimic.mit.edu/docs/iv/modules/).
- To access a table, you must reference the module it's contained in. For example, `hosp.labevents`, `ed.diagnosis`. If you aren't sure about the name of a table, you can use the `list_tables_in_module` function below to get a list of tables in a given module.
- All modules are not available yet. `hosp`, `icu`, `ed` and `note` should be fully functional.
- Queries can return large tables. You might want to look into processing them with chunks. At the very least, make sure you quit or restart the notebook when you're done working, in order to free up RAM for other users.

In [4]:
# Make sure these libraries are installed first!
# Run: pip install pandas sqlalchemy psycopg2

import pandas as pd
from sqlalchemy import create_engine

In [5]:
def execute_query(sql_query, user, password, host, port, database):
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

    try:
        df = pd.read_sql_query(sql_query, engine)
        return df
    finally:
        engine.dispose()

In [6]:
# Connection parameters. You shouldn't have to change them.
params = {
    'database': 'mimic_iv',
    'user': 'mimic_read_only_user',
    'password': 'mimiciv',
    'host': 'localhost',
    'port': '5432'
}

In [7]:
# Example SQL query
query = "SELECT * FROM hosp.labevents LIMIT 100;"

In [8]:
execute_query(query, **params)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,4136,10000904,28328117.0,38570510,51255,None,2180-10-09 17:00:00,2180-10-09 19:40:00,0,0.0,%,0.0,0.0,None,STAT,None
1,4137,10000904,28328117.0,38570510,51256,None,2180-10-09 17:00:00,2180-10-09 19:40:00,28,28.0,%,50.0,70.0,abnormal,STAT,None
2,4138,10000904,28328117.0,38570510,51265,None,2180-10-09 17:00:00,2180-10-09 17:45:00,247,247.0,K/uL,150.0,440.0,None,STAT,None
3,4139,10000904,28328117.0,38570510,51266,None,2180-10-09 17:00:00,2180-10-09 19:40:00,None,NaN,None,NaN,NaN,None,STAT,NORMAL.
4,4140,10000904,28328117.0,38570510,51267,None,2180-10-09 17:00:00,2180-10-09 19:40:00,None,NaN,None,NaN,NaN,None,STAT,NORMAL.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4231,10000935,NaN,36096610,51506,None,2182-05-11 09:25:00,2182-05-11 09:36:00,Clear,NaN,None,NaN,NaN,None,STAT,None
96,4232,10000935,NaN,36096610,51508,None,2182-05-11 09:25:00,2182-05-11 09:36:00,Yellow,NaN,None,NaN,NaN,None,STAT,None
97,4233,10000935,NaN,36096610,51512,None,2182-05-11 09:25:00,2182-05-11 09:47:00,FEW,NaN,None,NaN,NaN,None,STAT,None
98,4234,10000935,NaN,36096610,51514,None,2182-05-11 09:25:00,2182-05-11 09:36:00,NEG,NaN,mg/dL,0.2,1.0,None,STAT,None


In [9]:
def list_tables_in_module(module_name, user, password, host, port, database):
    list_tables_query = f"""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = '{module_name}'
    """
    
    return execute_query(list_tables_query, user, password, host, port, database)

In [10]:
list_tables_in_module('hosp', **params)

,table_name
0,d_icd_procedures
1,provider
2,diagnoses_icd
3,emar_detail
4,patients
5,admissions
6,drgcodes
7,emar
8,hcpcsevents
9,d_hcpcs


In [11]:
import json
def load_json(file):
    f = open(f"./data/{file}.json")
    df = pd.DataFrame(json.load(f))
    f.close()
    return df

In [12]:
#load_json("ckd_codes")
load_json("most_common_analyses")
#load_json("selected_analyses")

,label,itemid,event_count,percentage
0,Hematocrit,51221,3325700,2.8143027236030873
1,Creatinine,50912,3282278,2.7775577818271326
2,Platelet Count,51265,3216656,2.7220265633383085
3,Urea Nitrogen,51006,3189474,2.6990243753378937
4,Hemoglobin,51222,3188835,2.6984836352108883
...,...,...,...,...
910,"Uric Acid, Joint Fluid",51826,1,8.4622868075986638e-07
911,Hyperseg,52055,1,8.4622868075986638e-07
912,"Albumin, Stool",51927,1,8.4622868075986638e-07
913,Voided Specimen,52066,1,8.4622868075986638e-07
